In [159]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

sns.set_style("darkgrid")
sns.set_context("notebook")

Niektoré časti kódu boli inšpirované/použité z tohto python notebooku https://www.kaggle.com/code/docxian/it-salary-survey-eu-2020

## Bod 1 - stiahnutie datasetu...

In [160]:
df = pd.read_csv("dataset/IT Salary Survey EU 2020.csv")

## Bod 2
prozkoumejte jednotlivé atributy datové sady, jejich typ a hodnoty, kterých nabývají (počet hodnot, nejčastější hodnoty, rozsah hodnot atd.)

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df.describe()

Prevedieme string hodnoty na uppercase pre lepšie budúce čistenie

In [164]:
def clean_string(x):
    x = x.strip().lower().capitalize()
    return x

features_for_string_cleaning = ['City',
                                'Your main technology / programming language',
                                'Other technologies/programming languages you use often']

for f in features_for_string_cleaning:
    df[f] = df[f].fillna("0")
    df[f] = df[f].apply(clean_string)

## podukol 1 & 2 - popis jednotlivých atribútov datovej sady, typ a hodnota (počet hodnôt, najčastejšia hodnota, rozsah hodnôt) + rozloženie hodnôt jednotivých atribútov vykreslené pomocou vhodných grafov

### 1. Časová značka (Timestamp)

In [ ]:
df.Timestamp.describe()

### 2. Vek (Age)

In [ ]:
df.Age.describe()

In [ ]:
fig, axes = plt.subplots(ncols=1, nrows=2, figsize=(20, 18))

plot1 = sns.histplot(data=df.Age, ax=axes[0])

plot1.set_xlabel('vek', fontsize=16);
plot1.set_ylabel('počet', fontsize=16);
plot1.set_title('Vek IT pracovníkov ktorí vyplnili dotazník', fontsize=20)

for c in axes[0].containers:
    labels = [f'{(v.get_height()):.0f}' for v in c]
    axes[0].bar_label(c, labels=labels, label_type='edge')

plot2 = sm.qqplot_2samples(df.Age, df["Yearly brutto salary (without bonus and stocks) in EUR"], ax=axes[1])

axes[1].set_xlabel('vek', fontsize=16);
axes[1].set_ylabel('plat', fontsize=16);
axes[1].set_title('QQ-graf vek - plat', fontsize=20)

### 3. Pohlavie (Gender)

In [ ]:
df.Gender.describe()

In [ ]:
fig, axes = plt.subplots(figsize=(12, 8))

plot1 = sns.histplot(data=df.Gender, ax=axes)

axes.set_xlabel('', fontsize=16);
axes.set_ylabel('počet', fontsize=16);
axes.set_title('Pohlavie', fontsize=20)

axes.set_xticklabels(["Muž", "Žena", "Iné"])
for c in axes.containers:
    labels = [f'{(v.get_height()):.0f}' for v in c]
    axes.bar_label(c, labels=labels, label_type='edge')

### 4. Mesto (City)

In [ ]:
df.City.describe()

In [ ]:
# reduce levels
coll = "City"

df[coll].loc[df[coll]=='Bölingen'] = 'Boeblingen'
df[coll].loc[df[coll]=='Dusseldorf'] = 'Duesseldorf'
df[coll].loc[df[coll]=='Düsseldorf'] = 'Duesseldorf'
df[coll].loc[df[coll]=='Dusseldurf'] = 'Duesseldorf'
df[coll].loc[df[coll]=='Nürnberg'] = 'Nuremberg'
df[coll].loc[df[coll]=='Warsaw, Poland'] = 'Warsaw'
df[coll].loc[df[coll]=='Zürich'] = 'Zurich'

temp_count = df[coll].value_counts()

freq_min = 5
keep_levels = list(temp_count[temp_count.values>=freq_min].index)
df[coll] = df[coll].where(df[coll].isin(keep_levels), 'Other cities')


In [ ]:
# plot for reduced column
fig, axes = plt.subplots(figsize=(20, 8))

plot1 = sns.countplot(x=df.City, ax=axes, order=df.City.value_counts().index)

plot1.set_xlabel('', fontsize=16);
plot1.set_ylabel('', fontsize=16);
plot1.set_title('Počet IT pracovníkov z jednotlivých miest', fontsize=20)

#axes.set_xticklabels(["Muž", "Žena", "Iné"])
for c in axes.containers:
    labels = [f'{(v.get_height()):.0f}' for v in c]
    axes.bar_label(c, labels=labels, label_type='edge')
    

### 5. Pozícia (Position)

In [ ]:
df["Position "].describe()

In [174]:
# reduce levels
coll = "Position "

temp_count = df[coll].value_counts()

freq_min = 5
keep_levels = list(temp_count[temp_count.values>=freq_min].index)
df[coll] = df[coll].where(df[coll].isin(keep_levels), 'Other position')


In [ ]:
fig, axes = plt.subplots(figsize=(20, 12))

plot1 = sns.countplot(y=df["Position "], ax=axes, order=df["Position "].value_counts().index)

plot1.set_xlabel('', fontsize=16);
plot1.set_ylabel('', fontsize=16);
plot1.set_title('Počet IT pracovníkov na jednotlivých pozíciách', fontsize=20)

for c in axes.containers:
    labels = [f'{(v.get_width()):.0f}' for v in c]
    axes.bar_label(c, labels=labels, label_type='edge')

### 6. Celkový počet rokov skúseností / odpracovaných rokov (Total years of experience)

In [ ]:
coll = "Total years of experience"

df[coll].loc[df[coll]=='6 (not as a data scientist, but as a lab scientist)'] = '6'
df[coll].loc[df[coll]=='less than year'] = '1'
df[coll].loc[df[coll]=='15, thereof 8 as CTO'] = '15'
df[coll].loc[df[coll]=='1 (as QA Engineer) / 11 in total'] = '11'
df[coll].loc[df[coll]=='383'] = '0'
df[coll].loc[df[coll]=='1,5'] = '1.5'
df[coll].loc[df[coll]=='2,5'] = '2.5'
df[coll] = df[coll].astype(float)

In [ ]:
df["Total years of experience"].describe()

In [ ]:
fig, axes = plt.subplots(figsize=(20, 16))

plot1 = sns.countplot(y=df["Total years of experience"], ax=axes, order=df["Total years of experience"].value_counts().index)

plot1.set_xlabel('', fontsize=16);
plot1.set_ylabel('', fontsize=16);
plot1.set_title('Celkový počet počet rokov skúseností', fontsize=20)

for c in axes.containers:
    labels = [f'{(v.get_width()):.0f}' for v in c]
    axes.bar_label(c, labels=labels, label_type='edge')

### 7. Počet rokov skúseností / odpracovaných rokov v Nemecku (Total years of experience in Germany)

In [ ]:
coll = "Years of experience in Germany"

df[coll].loc[df[coll]=='0,3'] = '0.3'
df[coll].loc[df[coll]=='0,5'] = '0.5'
df[coll].loc[df[coll]=='1,5'] = '1.5'
df[coll].loc[df[coll]=='1,7'] = '1.7'
df[coll].loc[df[coll]=='2,5'] = '2.5'
df[coll].loc[df[coll]=='3,5'] = '3.5'
df[coll].loc[df[coll]=='4,5'] = '4.5'
df[coll].loc[df[coll]=='<1'] = '0.5'
df[coll].loc[df[coll]=='< 1'] = '0.5'
df[coll].loc[df[coll]=='3 months'] = '0.25'
df[coll].loc[df[coll]=='4 month'] = '0.33'
df[coll].loc[df[coll]=='4 (in Switzerland), 0 (in Germany)'] = '0'
df[coll].loc[df[coll]=='less than year'] = '0.5'
df[coll].loc[df[coll]=='⁰'] = '0'
df[coll].loc[df[coll]=='-'] = '0'
df[coll].loc[df[coll]=='6 (not as a data scientist, but as a lab scientist)'] = '6'
df[coll].loc[df[coll]=='3 (in Poland)'] = '0'
df[coll] = df[coll].astype(float)

In [ ]:
df["Years of experience in Germany"].describe()

In [ ]:
fig, axes = plt.subplots(figsize=(20, 16))

plot1 = sns.countplot(y=df["Years of experience in Germany"], ax=axes, order=df["Years of experience in Germany"].value_counts().index)

plot1.set_xlabel('', fontsize=16);
plot1.set_ylabel('', fontsize=16);
plot1.set_title('Počet rokov skúseností v Nemecku', fontsize=20)

for c in axes.containers:
    labels = [f'{(v.get_width()):.0f}' for v in c]
    axes.bar_label(c, labels=labels, label_type='edge')

### 8. Level skúsenosti (Seniority level)

In [ ]:
df["Seniority level"].describe()

In [183]:
coll = "Seniority level"

temp_count = df[coll].value_counts()

freq_min = 5
keep_levels = list(temp_count[temp_count.values>=freq_min].index)
df[coll] = df[coll].where(df[coll].isin(keep_levels), 'Other level')

In [ ]:
fig, axes = plt.subplots(figsize=(18, 12))

plot1 = sns.countplot(x=df["Seniority level"], ax=axes, order=df["Seniority level"].value_counts().index)

plot1.set_xlabel('', fontsize=16);
plot1.set_ylabel('', fontsize=16);
plot1.set_title('Level skúsenosti', fontsize=20)

for c in axes.containers:
    labels = [f'{(v.get_height()):.0f}' for v in c]
    axes.bar_label(c, labels=labels, label_type='edge')

### 9. Vaša hlavná technológia / programovací jazyk (Your main technology / programming language)

In [ ]:
df["Your main technology / programming language"].describe()

In [ ]:
df["Your main technology / programming language"].unique()

In [ ]:
coll = "Your main technology / programming language"

df[coll].loc[df[coll]=='Javascript'] = 'JavaScript'
df[coll].loc[df[coll]=='Javascript/es6'] = 'JavaScript'
df[coll].loc[df[coll]=='js'] = 'JavaScript'
df[coll].loc[df[coll]=='Js'] = 'JavaScript'
df[coll].loc[df[coll]=='Javascript'] = 'JavaScript'
df[coll].loc[df[coll]=='web apps'] = 'JavaScript'
df[coll].loc[df[coll]=='Java, js'] = 'Java'

temp_count = df[coll].value_counts()

freq_min = 5
keep_levels = list(temp_count[temp_count.values>=freq_min].index)
df[coll] = df[coll].where(df[coll].isin(keep_levels), 'Merged')

In [ ]:
fig, axes = plt.subplots(figsize=(8, 20))

plot1 = sns.countplot(y=df["Your main technology / programming language"], ax=axes, order=df["Your main technology / programming language"].value_counts().index)
#plot1 = sns.histplot(y=df["Your main technology / programming language"], ax=axes)

plot1.set_xlabel('', fontsize=16);
plot1.set_ylabel('', fontsize=16);
plot1.set_title('Vaša hlavná technológia / programovací jazyk', fontsize=20)

for c in axes.containers:
    labels = [f'{(v.get_width()):.0f}' for v in c]
    axes.bar_label(c, labels=labels, label_type='edge')

### 10. Ďalšie technológie / programovacie jazyky ktoré často používate (Other technologies/programming languages you use often)

In [ ]:
df["Other technologies/programming languages you use often"].describe()

In [ ]:
df["Other technologies/programming languages you use often"].unique()

In [ ]:
coll = "Your main technology / programming language"

df[coll].loc[df[coll]=='Javascript'] = 'JavaScript'
df[coll].loc[df[coll]=='Javascript/es6'] = 'JavaScript'
df[coll].loc[df[coll]=='js'] = 'JavaScript'
df[coll].loc[df[coll]=='Js'] = 'JavaScript'
df[coll].loc[df[coll]=='Javascript'] = 'JavaScript'
df[coll].loc[df[coll]=='web apps'] = 'JavaScript'
df[coll].loc[df[coll]=='Java, js'] = 'Java'

temp_count = df[coll].value_counts()

freq_min = 5
keep_levels = list(temp_count[temp_count.values>=freq_min].index)
df[coll] = df[coll].where(df[coll].isin(keep_levels), 'Merged')

### 11. Ročná hrubá mzda (bez bonusov a akcií) v eurách - (yearly brutto salary (without bonus and stocks) in EUR)

In [ ]:
df["Yearly brutto salary (without bonus and stocks) in EUR"].describe()

In [ ]:
fig = px.box(df, x="Yearly brutto salary (without bonus and stocks) in EUR", log_x=True,
    labels={"Yearly brutto salary (without bonus and stocks) in EUR": "Ročná hrubá mzda bez bonusov v EUR"})
#fig.show()
fig

### 12. Ročný bonus + akcie v EUR (yearly bonus + stocks in EUR)

In [ ]:
df["Yearly bonus + stocks in EUR"].describe()

In [ ]:
fig = px.box(df, x="Yearly bonus + stocks in EUR", log_x=True,
    labels={"Yearly bonus + stocks in EUR": "Ročný bonus + akcie v EUR"})
fig.show()

### 13. Ročná hrubá mzda (bez bonusu a akcií) pred rokom. Odpovedajte iba ak zostávate v rovnakej krajine. 
### (Annual brutto salary (without bonus and stocks) one year ago. Only answer if staying in the same country)

In [ ]:
df["Annual brutto salary (without bonus and stocks) one year ago. Only answer if staying in the same country"].describe()

In [ ]:
fig = px.box(df, x="Annual brutto salary (without bonus and stocks) one year ago. Only answer if staying in the same country", log_x=True,
    labels={"Annual brutto salary (without bonus and stocks) one year ago. Only answer if staying in the same country": 
    "Ročná hrubá mzda (bez bonusu a akcií) pred rokom"})
fig.show()

### 14. Ročný bonus + akcie pred rokom. Odpovedajte iba ak zostávate v rovnakej krajine.


### (Annual bonus+stocks one year ago. Only answer if staying in same country)

In [ ]:
df["Annual bonus+stocks one year ago. Only answer if staying in same country"].describe()

In [ ]:
fig = px.box(df, x="Annual bonus+stocks one year ago. Only answer if staying in same country", log_x=True,
    labels={"Annual bonus+stocks one year ago. Only answer if staying in same country": 
    "Ročný bonus + akcie pred rokom"})
fig.show()

### 15. Počet dní pracovného voľna (Number of vacation days)

In [ ]:
df["Number of vacation days"].describe()

In [ ]:
coll = 'Number of vacation days'

df[coll].loc[df[coll]=='30 IN CONTRACT (BUT THEORETICALLY UNLIMITED)'] = 'UNLIMITED'
df[coll].loc[df[coll]=='23+'] = '23'
df[coll].loc[df[coll]=='(NO IDEA)'] = '_MISSING_'
df[coll].loc[df[coll]=='24 LABOUR DAYS'] = '24'
df[coll].loc[df[coll]=='~25'] = '25'
df[coll].loc[df[coll]=='365'] = 'UNLIMITED'

In [ ]:
fig, axes = plt.subplots(figsize=(20, 16))

plot1 = sns.lineplot(data=df, y="Number of vacation days", x="Age",
                    ax=axes)

plot1.set_xlabel('', fontsize=16);
plot1.set_ylabel('', fontsize=16);
plot1.set_title('Počet dní pracovného voľna', fontsize=20)

for c in axes.containers:
    labels = [f'{(v.get_width()):.0f}' for v in c]
    axes.bar_label(c, labels=labels, label_type='edge')

### 16. Ako zamestnaný (Employment status)

In [ ]:
df["Employment status"].describe()

In [204]:
coll = "Employment status"

temp_count = df[coll].value_counts()

freq_min = 5
keep_levels = list(temp_count[temp_count.values>=freq_min].index)
df[coll] = df[coll].where(df[coll].isin(keep_levels), 'Other')

In [ ]:
fig, axes = plt.subplots(figsize=(12, 6))

plot1 = sns.countplot(y=df["Employment status"],
                    ax=axes, order=df["Employment status"].value_counts().index)

plot1.set_xlabel('', fontsize=16);
plot1.set_ylabel('', fontsize=16);
plot1.set_title('Typ pracovného úväzku', fontsize=20)

for c in axes.containers:
    labels = [f'{(v.get_width()):.0f}' for v in c]
    axes.bar_label(c, labels=labels, label_type='edge')

### 17. Doba trvania kontraktu (Сontract duration)

In [ ]:
df["Сontract duration"].describe()

In [ ]:
fig, axes = plt.subplots(figsize=(12, 8))

plot1 = sns.histplot(x=df["Сontract duration"],
                    ax=axes)

plot1.set_xlabel('', fontsize=16);
plot1.set_ylabel('', fontsize=16);
plot1.set_title('Doba trvania kontraktu', fontsize=20)

for c in axes.containers:
    labels = [f'{(v.get_height()):.0f}' for v in c]
    axes.bar_label(c, labels=labels, label_type='edge')

### 18. Hlavný jazyk v práci (Main language at work)

In [ ]:
df["Main language at work"].describe()

In [ ]:
df["Main language at work"].unique()

In [ ]:
coll = "Main language at work"

df[coll].loc[df[coll]=='both'] = 'English and German'
df[coll].loc[df[coll]=='Deuglisch'] = 'English and German'
df[coll].loc[df[coll]=='Русский'] = 'Russian'
df[coll].loc[df[coll]=='50/50'] = 'English and German'

temp_count = df[coll].value_counts()

freq_min = 5
keep_levels = list(temp_count[temp_count.values>=freq_min].index)
df[coll] = df[coll].where(df[coll].isin(keep_levels), 'Other language combinations')

In [ ]:
fig, axes = plt.subplots(figsize=(14, 6))

plot1 = sns.histplot(x=df["Main language at work"],
                    ax=axes)

plot1.set_xlabel('', fontsize=16);
plot1.set_ylabel('', fontsize=16);
plot1.set_title('Hlavný jazyk v práci', fontsize=20)

for c in axes.containers:
    labels = [f'{(v.get_height()):.0f}' for v in c]
    axes.bar_label(c, labels=labels, label_type='edge')

### 19. Veľkosť spoločnosti (Company size)

In [ ]:
df["Company size"].describe().unique

In [ ]:
fig, axes = plt.subplots(figsize=(16, 12))

df["Company size"] = pd.Categorical(df["Company size"], ['up to 10','11-50','51-100','101-1000', '1000+'])

#plot1 = sns.lineplot(data=df["Company size"], ax=axes)

plot1 = sns.histplot(data=df, x="Company size",
                    ax=axes)

plot1.set_xlabel('', fontsize=16);
plot1.set_ylabel('', fontsize=16);
plot1.set_title('Veľkosť spoločnosti', fontsize=20)

for c in axes.containers:
    labels = [f'{(v.get_height()):.0f}' for v in c]
    axes.bar_label(c, labels=labels, label_type='edge')

### 20. Typ spoločnosti (Company type)

In [ ]:
df["Company type"].describe()

In [ ]:
df["Company type"].unique()

In [ ]:
# reduce levels
coll = "Company type"

df[coll].loc[df[coll]=='e-commerce'] = 'E-Commerce'
df[coll].loc[df[coll]=='e-commerce'] = 'e-commerce'

temp_count = df[coll].value_counts()

freq_min = 5
keep_levels = list(temp_count[temp_count.values>=freq_min].index)
df[coll] = df[coll].where(df[coll].isin(keep_levels), 'Merged')


In [ ]:
fig, axes = plt.subplots(figsize=(12, 8))


plot1 = sns.countplot(data=df, x="Company type", order=df["Company type"].value_counts().index,
                    ax=axes)

plot1.set_xlabel('', fontsize=16);
plot1.set_ylabel('', fontsize=16);
plot1.set_title('Typ spoločnosti', fontsize=20)

for c in axes.containers:
    labels = [f'{(v.get_height()):.0f}' for v in c]
    axes.bar_label(c, labels=labels, label_type='edge')

### 21. Stratili ste svoju prácu kvôli pandémií koronavírusu (Have you lost your job due to the coronavirus outbreak?)

In [ ]:
df["Have you lost your job due to the coronavirus outbreak?"].describe()

In [ ]:
df["Have you lost your job due to the coronavirus outbreak?"].unique()

In [ ]:
# reduce levels
coll = "Have you lost your job due to the coronavirus outbreak?"

df[coll].loc[df[coll]=="i didn't but will be looking for new one because of covid"] = 'No'
df[coll].loc[df[coll]=='kurzarbeitzeit for 1.5 months'] = 'No'
df[coll].loc[df[coll]=='Have been a freelancer at the beginning of year'] = 'No'
df[coll].loc[df[coll]=='No, but there was a salary cut at 10% for 3 months and then at 5% for further 3 months'] = 'No'
df[coll].loc[df[coll]=='yes but found a new one with better pay / perks'] = 'Yes'
df[coll].loc[df[coll]=='Leads and project inquiries have slowed down'] = 'No'

temp_count = df[coll].value_counts()

freq_min = 5
keep_levels = list(temp_count[temp_count.values>=freq_min].index)
df[coll] = df[coll].where(df[coll].isin(keep_levels), 'Other reason')


In [ ]:
fig, axes = plt.subplots(figsize=(16, 10))

plot1 = sns.countplot(data=df, y="Have you lost your job due to the coronavirus outbreak?",
                order=df["Have you lost your job due to the coronavirus outbreak?"].value_counts().index, ax=axes)

plot1.set_xlabel('', fontsize=16);
plot1.set_ylabel('', fontsize=16);
plot1.set_title('Stratili ste svoju prácu kvôli pandémií koronavírusu?', fontsize=20)

for c in axes.containers:
    labels = [f'{(v.get_width()):.0f}' for v in c]
    axes.bar_label(c, labels=labels, label_type='edge')

### 22. Boli ste nútený mať kratšie pracovné týždne (Kurzarbeit)? Ak áno, koľko to bolo hodín za týždeň? 
### (Have you been forced to have a shorter working week (Kurzarbeit)? If yes, how many hours per week)

In [ ]:
df["Have you been forced to have a shorter working week (Kurzarbeit)? If yes, how many hours per week"].describe()

In [ ]:
df["Have you been forced to have a shorter working week (Kurzarbeit)? If yes, how many hours per week"].plot.line()

In [ ]:
fig, axes = plt.subplots(figsize=(26, 10))

plot1 = sns.lineplot(data=df, y="Have you been forced to have a shorter working week (Kurzarbeit)? If yes, how many hours per week",
        x="Age",
        #y=df["Have you been forced to have a shorter working week (Kurzarbeit)? If yes, how many hours per week"].value_counts(),
        ax=axes)

plot1.set_xlabel('Vek', fontsize=16);
plot1.set_ylabel('Počet hodín na kratšom týždni', fontsize=16);

In [ ]:


fig = px.box(df, x="Yearly bonus + stocks in EUR", log_x=True,
    labels={"Yearly bonus + stocks in EUR": "Ročný bonus + akcie v EUR"})
fig.show()

In [ ]:
fig, axes = plt.subplots(figsize=(26, 10))

plot1 = sns.histplot(data=df, x="Have you been forced to have a shorter working week (Kurzarbeit)? If yes, how many hours per week",
        ax=axes)

#order=df["Have you been forced to have a shorter working week (Kurzarbeit)? If yes, how many hours per week"].value_counts().index, ax=axes)                
""" 
plot1.set_xlabel('', fontsize=16);
plot1.set_ylabel('', fontsize=16);
plot1.set_title('Boli ste nútený máť kratšie pracovné týždne (Kurzarbeit)? Ak áno, koľko to bolo hodín za týždeň?', fontsize=20)

for c in axes.containers:
    labels = [f'{(v.get_height()):.0f}' for v in c]
    axes.bar_label(c, labels=labels, label_type='edge') """

#df["Have you been forced to have a shorter working week (Kurzarbeit)? If yes, how many hours per week"].plot.line()


### 23. Dostali ste nejakú dodatočnú podporu od svojho zamestnávateľa z dôvodu práce z domu? ak áno, ako veľa v roku 2020 v EUR?
### Have you received additional monetary support from your employer due to Work From Home? If yes, how much in 2020 in EUR

In [ ]:
df["Have you received additional monetary support from your employer due to Work From Home? If yes, how much in 2020 in EUR"].describe()

In [ ]:
fig, axes = plt.subplots(figsize=(16, 20))

plot1 = sns.countplot(data=df, y="Have you received additional monetary support from your employer due to Work From Home? If yes, how much in 2020 in EUR",
        ax=axes)
#order=df["Have you been forced to have a shorter working week (Kurzarbeit)? If yes, how many hours per week"].value_counts().index, ax=axes)                

plot1.set_xlabel('', fontsize=16);
plot1.set_ylabel('', fontsize=16);
plot1.set_title('Dostali ste nejakú dodatočnú podporu od svojho zamestnávateľa z dôvodu práce z domu? ak áno, ako veľa v roku 2020 v EUR?', fontsize=20)

for c in axes.containers:
    labels = [f'{(v.get_width()):.0f}' for v in c]
    axes.bar_label(c, labels=labels, label_type='edge')

In [ ]:
plt.figure(figsize=(10,10))

# Select only numeric columns
numeric_df = df.select_dtypes(include=['float64', 'int64'])
corr = numeric_df.corr()

ax = sns.heatmap(
    corr,
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True, annot=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
)
ax.set_yticklabels(
    ax.get_yticklabels(),
    rotation=45,
);

In [ ]:
plt.figure(figsize=(10,10))

##corr = df["Age"].corr(df["Gender"])
#corr = df.corr()["Age"]

newdf = df[["Age", "Total years of experience"]].copy()

df_dummies = pd.get_dummies(df["Employment status"])
del df_dummies[df_dummies.columns[-1]]
newdf = pd.concat([newdf, df_dummies], axis=1)
#del newdf['color_head']

corr = newdf.corr()

ax = sns.heatmap(
    corr,
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True, annot=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
)
ax.set_yticklabels(
    ax.get_yticklabels(),
    rotation=45,
);

In [ ]:
plt.figure(figsize=(10,10))

##corr = df["Age"].corr(df["Gender"])
#corr = df.corr()["Age"]

newdf = df[["Age", "Total years of experience", "Yearly brutto salary (without bonus and stocks) in EUR"]].copy()

newdf["Total years of experience"].loc[newdf["Total years of experience"]=='6 (not as a data scientist, but as a lab scientist)'] = '6'
newdf["Total years of experience"].loc[newdf["Total years of experience"]=='less than year'] = '1'
newdf["Total years of experience"].loc[newdf["Total years of experience"]=='15, thereof 8 as CTO'] = '15'
newdf["Total years of experience"].loc[newdf["Total years of experience"]=='1 (as QA Engineer) / 11 in total'] = '11'
newdf["Total years of experience"].loc[newdf["Total years of experience"]=='383'] = '0'
newdf["Total years of experience"].loc[newdf["Total years of experience"]=='1,5'] = '1.5'
newdf["Total years of experience"].loc[newdf["Total years of experience"]=='2,5'] = '2.5'
newdf["Total years of experience"] = newdf["Total years of experience"].astype(float)

#corr = newdf["Age"].corr(newdf["Total years of experience"])
corr = newdf.corr()

ax = sns.heatmap(
    corr,
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True, annot=True
)
ax.set_xticklabels(
    ["Vek", "Roky skúseností", "Hrubá mzda bez bonusov"],
    rotation=45,
    horizontalalignment='right'
)

ax.set_yticklabels(
    ["Vek", "Roky skúseností", "Hrubá mzda bez bonusov"],
    rotation=45,
)

## Bod 3 - datové sady

Všeobecné čistenie pre obe datové sady

In [ ]:
df = pd.read_csv("dataset/IT Salary Survey EU 2020.csv")

def clean_string(x):
    x = x.strip().lower().capitalize()
    return x

features_for_string_cleaning = ['City',
                                'Your main technology / programming language',
                                'Other technologies/programming languages you use often']

for f in features_for_string_cleaning:
    df[f] = df[f].fillna("0")
    df[f] = df[f].apply(clean_string)

# reduce levels
coll = "City"

df[coll].loc[df[coll]=='Bölingen'] = 'Boeblingen'
df[coll].loc[df[coll]=='Dusseldorf'] = 'Duesseldorf'
df[coll].loc[df[coll]=='Düsseldorf'] = 'Duesseldorf'
df[coll].loc[df[coll]=='Dusseldurf'] = 'Duesseldorf'
df[coll].loc[df[coll]=='Nürnberg'] = 'Nuremberg'
df[coll].loc[df[coll]=='Warsaw, Poland'] = 'Warsaw'
df[coll].loc[df[coll]=='Zürich'] = 'Zurich'

temp_count = df[coll].value_counts()

freq_min = 5
keep_levels = list(temp_count[temp_count.values>=freq_min].index)
df[coll] = df[coll].where(df[coll].isin(keep_levels), 'Other cities')

coll = "Position "

temp_count = df[coll].value_counts()

freq_min = 5
keep_levels = list(temp_count[temp_count.values>=freq_min].index)
df[coll] = df[coll].where(df[coll].isin(keep_levels), 'Other position')

coll = "Total years of experience"

df[coll].loc[df[coll]=='6 (not as a data scientist, but as a lab scientist)'] = '6'
df[coll].loc[df[coll]=='less than year'] = '1'
df[coll].loc[df[coll]=='15, thereof 8 as CTO'] = '15'
df[coll].loc[df[coll]=='1 (as QA Engineer) / 11 in total'] = '11'
df[coll].loc[df[coll]=='383'] = '0'
df[coll].loc[df[coll]=='1,5'] = '1.5'
df[coll].loc[df[coll]=='2,5'] = '2.5'
df[coll] = df[coll].astype(float)

coll = "Years of experience in Germany"

df[coll].loc[df[coll]=='0,3'] = '0.3'
df[coll].loc[df[coll]=='0,5'] = '0.5'
df[coll].loc[df[coll]=='1,5'] = '1.5'
df[coll].loc[df[coll]=='1,7'] = '1.7'
df[coll].loc[df[coll]=='2,5'] = '2.5'
df[coll].loc[df[coll]=='3,5'] = '3.5'
df[coll].loc[df[coll]=='4,5'] = '4.5'
df[coll].loc[df[coll]=='<1'] = '0.5'
df[coll].loc[df[coll]=='< 1'] = '0.5'
df[coll].loc[df[coll]=='3 months'] = '0.25'
df[coll].loc[df[coll]=='4 month'] = '0.33'
df[coll].loc[df[coll]=='4 (in Switzerland), 0 (in Germany)'] = '0'
df[coll].loc[df[coll]=='less than year'] = '0.5'
df[coll].loc[df[coll]=='⁰'] = '0'
df[coll].loc[df[coll]=='-'] = '0'
df[coll].loc[df[coll]=='6 (not as a data scientist, but as a lab scientist)'] = '6'
df[coll].loc[df[coll]=='3 (in Poland)'] = '0'
df[coll] = df[coll].astype(float)

coll = "Your main technology / programming language"

df[coll].loc[df[coll]=='Javascript'] = 'JavaScript'
df[coll].loc[df[coll]=='Javascript/es6'] = 'JavaScript'
df[coll].loc[df[coll]=='js'] = 'JavaScript'
df[coll].loc[df[coll]=='Js'] = 'JavaScript'
df[coll].loc[df[coll]=='Javascript'] = 'JavaScript'
df[coll].loc[df[coll]=='web apps'] = 'JavaScript'
df[coll].loc[df[coll]=='Java, js'] = 'Java'

temp_count = df[coll].value_counts()

freq_min = 5
keep_levels = list(temp_count[temp_count.values>=freq_min].index)
df[coll] = df[coll].where(df[coll].isin(keep_levels), 'Merged')

coll = 'Number of vacation days'

df[coll].loc[df[coll]=='30 IN CONTRACT (BUT THEORETICALLY UNLIMITED)'] = 'UNLIMITED'
df[coll].loc[df[coll]=='23+'] = '23'
df[coll].loc[df[coll]=='(NO IDEA)'] = '_MISSING_'
df[coll].loc[df[coll]=='24 LABOUR DAYS'] = '24'
df[coll].loc[df[coll]=='~25'] = '25'
df[coll].loc[df[coll]=='365'] = 'UNLIMITED'

coll = "Main language at work"

df[coll].loc[df[coll]=='both'] = 'English and German'
df[coll].loc[df[coll]=='Deuglisch'] = 'English and German'
df[coll].loc[df[coll]=='Русский'] = 'Russian'
df[coll].loc[df[coll]=='50/50'] = 'English and German'

temp_count = df[coll].value_counts()

freq_min = 5
keep_levels = list(temp_count[temp_count.values>=freq_min].index)
df[coll] = df[coll].where(df[coll].isin(keep_levels), 'Other language combinations')

coll = "Company type"

df[coll].loc[df[coll]=='e-commerce'] = 'E-Commerce'
df[coll].loc[df[coll]=='e-commerce'] = 'e-commerce'

temp_count = df[coll].value_counts()

freq_min = 5
keep_levels = list(temp_count[temp_count.values>=freq_min].index)
df[coll] = df[coll].where(df[coll].isin(keep_levels), 'Merged')

# reduce levels
coll = "Have you lost your job due to the coronavirus outbreak?"

df[coll].loc[df[coll]=="i didn't but will be looking for new one because of covid"] = 'No'
df[coll].loc[df[coll]=='kurzarbeitzeit for 1.5 months'] = 'No'
df[coll].loc[df[coll]=='Have been a freelancer at the beginning of year'] = 'No'
df[coll].loc[df[coll]=='No, but there was a salary cut at 10% for 3 months and then at 5% for further 3 months'] = 'No'
df[coll].loc[df[coll]=='yes but found a new one with better pay / perks'] = 'Yes'
df[coll].loc[df[coll]=='Leads and project inquiries have slowed down'] = 'No'

temp_count = df[coll].value_counts()

freq_min = 5
keep_levels = list(temp_count[temp_count.values>=freq_min].index)
df[coll] = df[coll].where(df[coll].isin(keep_levels), 'Other reason')

In [ ]:
#transformace kategorických na numerické
df_dis = df[[
    "Main language at work",
    "Company size",
    "Company type",
    "Age"
]]

df_dis["Company size"] = df_dis["Company size"].replace(['up to 10','11-50','51-100','101-1000','1000+'], [0, 1, 2, 3, 4])
df_dis["Company type"] = df_dis["Company type"].replace(['Product','Startup','Consulting / Agency','Merged','Bank'], [0, 1, 2, 3, 4])
df_dis["Main language at work"] = df_dis["Main language at work"].replace(['English','German','English and German','Other language combinations','Russian'], [0, 1, 2, 3, 4])

df_dis.dropna(inplace=True)

df_dis.head(50)

df_dis.head(50).to_csv("dis.csv")

In [ ]:
#diskretizace numerických atrib na kategorické
df_cat = df[[
    "City",
    "Yearly brutto salary (without bonus and stocks) in EUR",
    "Age"
]]

df_cat.dropna(inplace=True)

df_cat.head(50).to_csv("cat.csv")